In [16]:
import mysql.connector
import pandas as pd
pd.set_option('display.max_columns', None)

# Connect to the MySQL database
db = mysql.connector.connect(
    host="localhost",
    user="root", 
    password="weatherC",
    database="GlobalWeatherDB"
)

In [17]:
def displayResults(cursor, results):
    columns = [i[0] for i in cursor.description]
    df = pd.DataFrame(results, columns=columns)
    return df

In [18]:
cursor = db.cursor()
query = "Describe WeatherData;"
cursor.execute(query)

results = cursor.fetchall()
cursor.close()
print(displayResults(cursor, results))

                           Field           Type Null Key Default Extra
0                        country   varchar(100)  YES        None      
1                  location_name   varchar(100)  YES        None      
2                       latitude  decimal(10,8)  YES        None      
3                      longitude  decimal(11,8)  YES        None      
4                       timezone    varchar(50)  YES        None      
5             last_updated_epoch         bigint  YES        None      
6                   last_updated       datetime  YES        None      
7            temperature_celsius   decimal(5,2)  YES        None      
8         temperature_fahrenheit   decimal(5,2)  YES        None      
9                 condition_text   varchar(100)  YES        None      
10                wind_speed_mph   decimal(7,2)  YES        None      
11                wind_speed_kph   decimal(7,2)  YES        None      
12                   wind_degree            int  YES        None      
13    

Retrieve the top 5 locations with the highest temperatures or
lowest precipitation.

In [19]:
cursor = db.cursor()
query = "SELECT location_name, MAX(temperature_celsius) AS max_temperature FROM WeatherData GROUP BY location_name ORDER BY max_temperature DESC LIMIT 5;"
cursor.execute(query)

results = cursor.fetchall()
cursor.close()
print(displayResults(cursor, results))

  location_name max_temperature
0   Kuwait City           49.20
1       Baghdad           49.10
2      Djibouti           47.10
3        Riyadh           46.30
4          Doha           46.20


Retrieve all records for a specific date or condition (e.g., temperature >= 35°C, precipitation >= 100 mm).

In [20]:
cursor = db.cursor()
query = "SELECT * FROM WeatherData WHERE temperature_celsius > 35 OR precipitation_mm >= 100;"
cursor.execute(query)
results = cursor.fetchall()
cursor.close()
print(displayResults(cursor, results))

           country location_name     latitude     longitude         timezone  \
0          Bahrain        Manama  26.24000000   50.58000000     Asia/Bahrain   
1       Bangladesh         Dhaka  23.72000000   90.41000000       Asia/Dhaka   
2         Cambodia    Phnom Penh  11.55000000  104.92000000  Asia/Phnom_Penh   
3             Chad     N'djamena  12.11000000   15.05000000    Africa/Douala   
4            India     New Delhi  28.60000000   77.20000000     Asia/Kolkata   
...            ...           ...          ...           ...              ...   
2379         Sudan      Khartoum  15.58810000   32.53420000  Africa/Khartoum   
2380          Iraq       Baghdad  33.33860000   44.39390000     Asia/Baghdad   
2381         Qatar          Doha  25.28670000   51.53330000       Asia/Qatar   
2382  Saudi Arabia        Riyadh  24.64080000   46.77280000      Asia/Riyadh   
2383         Sudan      Khartoum  15.58810000   32.53420000  Africa/Khartoum   

      last_updated_epoch        last_up

Perform a group by operation (e.g., average temperature by country, total precipitation by region).

In [21]:
cursor = db.cursor()
query = '''
SELECT location_name, AVG(temperature_celsius) AS average_temperature
FROM WeatherData
GROUP BY location_name;
'''
cursor.execute(query)
results = cursor.fetchall()
cursor.close()
print(displayResults(cursor, results))

            location_name average_temperature
0                   Kabul           27.441060
1                  Tirana           30.376159
2                 Algiers           28.769333
3        Andorra La Vella           18.373510
4                  Luanda           25.861589
..                    ...                 ...
243    City Of San Marino           31.300000
244             Ar Riyadh           45.000000
245  Krasnyy Turkmenistan           37.800000
246              -Kingdom           29.700000
247                 Airai           27.580328

[248 rows x 2 columns]


In [22]:
cursor = db.cursor()
query = '''
SELECT location_name, AVG(precipitation_mm) AS average_precip_mm
FROM WeatherData
GROUP BY location_name;
'''
cursor.execute(query)
results = cursor.fetchall()
cursor.close()
print(displayResults(cursor, results))

            location_name average_precip_mm
0                   Kabul          0.057947
1                  Tirana          0.174636
2                 Algiers          0.004267
3        Andorra La Vella          0.188609
4                  Luanda          0.000199
..                    ...               ...
243    City Of San Marino          0.000000
244             Ar Riyadh          0.000000
245  Krasnyy Turkmenistan          0.000000
246              -Kingdom          0.000000
247                 Airai          0.516393

[248 rows x 2 columns]


In [32]:
from pymongo import MongoClient

# Connect to the MongoDB server
client = MongoClient('mongodb://localhost:27017/')  # Update with your MongoDB URI if necessary

# Access the database and collection
db = client.WeatherData
collection = db.weather_data

# 1. Retrieve all records for a specific continent or month
records_condition = list(collection.find(
    {
    '$or': [
        { 'country': 'India' },
    ]
}
))

df_condition = pd.DataFrame(records_condition)
print("Records matching specific condition:")
print(df_condition)

# 2. Retrieve the top 3 hottest locations
top_hottest = list(collection.find().sort('temperature_celsius', -1).limit(3))
df_top_hottest = pd.DataFrame(top_hottest)
print("\nTop 3 hottest locations:")
print(df_top_hottest)

# 3. Retrieve the top 3 days with the highest precipitation
top_precipitation = list(collection.find().sort('precipitation_mm', -1).limit(3))
df_top_precipitation = pd.DataFrame(top_precipitation)
print("\nTop 3 days with highest precipitation:")
print(df_top_precipitation)

Records matching specific condition:
                          _id country location_name  latitude  longitude  \
0    671185362a4b3e29b2d4dad6   India     New Delhi      28.6       77.2   
1    671185362a4b3e29b2d4dba0   India     New Delhi      28.6       77.2   
2    671185362a4b3e29b2d4dc89   India     New Delhi      28.6       77.2   
3    671185362a4b3e29b2d4dd28   India     New Delhi      28.6       77.2   
4    671185362a4b3e29b2d4dddf   India     New Delhi      28.6       77.2   
..                        ...     ...           ...       ...        ...   
144  671185372a4b3e29b2d5499a   India     New Delhi      28.6       77.2   
145  671185372a4b3e29b2d54a4e   India     New Delhi      28.6       77.2   
146  671185372a4b3e29b2d54b12   India     New Delhi      28.6       77.2   
147  671185372a4b3e29b2d54bdc   India     New Delhi      28.6       77.2   
148  671185372a4b3e29b2d54c9e   India     New Delhi      28.6       77.2   

         timezone  last_updated_epoch      last_up